In [2]:
import torch

from transformers import AutoTokenizer

#加载tokenizer
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## 加载并处理数据集

In [3]:
from datasets import load_dataset

#加载数据集
dataset = load_dataset(path='lansinuote/ChnSentiCorp')

#编码
f = lambda x: tokenizer(
    x['text'], truncation=True, max_length=30, return_token_type_ids=False)
dataset = dataset.map(f, remove_columns=['text', 'label'])

#过滤句子长度
f = lambda x: len(x['input_ids']) >= 30
dataset = dataset.filter(f)


#重置label字段
def f(data):
    #定义第15个字为label
    data['label'] = data['input_ids'][15]

    #替换句子中的第15个字为mask
    data['input_ids'][15] = tokenizer.mask_token_id

    return data


dataset = dataset.map(f)

#设置数据类型
dataset.set_format('pt')

dataset, dataset['train'][0]

Map: 100%|██████████| 1157/1157 [00:00<00:00, 22762.09 examples/s]


(DatasetDict({
     train: Dataset({
         features: ['input_ids', 'attention_mask', 'label'],
         num_rows: 9286
     })
     validation: Dataset({
         features: ['input_ids', 'attention_mask', 'label'],
         num_rows: 1158
     })
     test: Dataset({
         features: ['input_ids', 'attention_mask', 'label'],
         num_rows: 1157
     })
 }),
 {'input_ids': tensor([ 101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221,
          3175,  912, 8024,  103, 4510, 1220, 2820, 3461, 4684, 2970, 1168, 6809,
          3862, 6804, 8024, 1453, 1741,  102]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1]),
  'label': tensor(3300)})

## 定义loader

In [4]:
loader = torch.utils.data.DataLoader(dataset['train'],
                                     batch_size=8,
                                     shuffle=True,
                                     drop_last=True)

data = next(iter(loader))

for k, v in data.items():
    print(k, v.shape)

len(loader)

input_ids torch.Size([8, 30])
attention_mask torch.Size([8, 30])
label torch.Size([8])


1160

## 数据样例

In [5]:
#查看数据样例
for q, a in zip(data['input_ids'], data['label']):
    print(tokenizer.decode(q))
    print(tokenizer.decode(a))
    print('==============')

[CLS] 配 置 一 流 ， 在 我 [UNK] 拷 贝 五 分 钟 的 [MASK] 件 ， 这 台 机 只 要 30 秒 就 搞 掂 ， [SEP]
文
[CLS] 如 果 对 于 日 本 这 个 国 家 仍 仅 仅 停 [MASK] 在 书 本 知 识 的 话 ， 那 么 这 本 书 [SEP]
留
[CLS] 一 直 都 想 给 儿 子 买 国 内 的 科 普 读 [MASK] ， 但 是 都 没 有 买 ， 原 因 是 说 教 [SEP]
物
[CLS] 服 务 很 热 情 ， 房 间 温 度 正 好 ， 隔 [MASK] 效 果 好 ， 地 理 位 置 优 越 ， 性 价 [SEP]
音
[CLS] 从 开 始 读 关 于 家 庭 教 育 的 书 ， 到 [MASK] 在 也 不 知 读 了 多 少 本 。 但 至 今 [SEP]
现
[CLS] 偶 觉 得 这 句 真 的 好 有 道 理 。 道 出 [MASK] 我 的 心 声 啊 ！ [UNK] 出 租 车 就 这 样 [SEP]
了
[CLS] 散 热 有 些 差 ， 可 能 [UNK] 的 通 病 。 右 [MASK] 面 有 时 热 的 太 过 分 。 价 格 降 的 [SEP]
下
[CLS] 坐 [UNK] 时 问 司 机 这 家 酒 店 怎 么 样 " [MASK] 老 酒 店, 人 气 旺, 服 务 好, 众 [SEP]
,


## 定义下游任务模型（Downstream Tasks by Pytorch)

In [6]:
#定义模型
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()

        #加载预训练模型
        from transformers import AutoModel
        self.pretrained = AutoModel.from_pretrained(
            'google-bert/bert-base-chinese')

        self.fc = torch.nn.Linear(in_features=768,
                                  out_features=tokenizer.vocab_size)

    def forward(self, input_ids, attention_mask, label=None):
        #使用预训练模型抽取数据特征
        with torch.no_grad():
            last_hidden_state = self.pretrained(
                input_ids=input_ids,
                attention_mask=attention_mask).last_hidden_state

        #取第15个词的特征向量
        last_hidden_state = last_hidden_state[:, 15]

        #对抽取的特征只取第一个字的结果做分类即可
        out = self.fc(last_hidden_state).softmax(dim=1)

        #计算loss
        loss = None
        if label is not None:
            loss = torch.nn.functional.cross_entropy(out, label)

        return loss, out


model = Model()

model(**data)

(tensor(9.9584, grad_fn=<NllLossBackward0>),
 tensor([[6.3790e-05, 4.9582e-05, 3.0027e-05,  ..., 4.5211e-05, 1.3296e-05,
          7.4145e-05],
         [6.3213e-05, 4.2645e-05, 3.7453e-05,  ..., 2.7083e-05, 1.8871e-05,
          4.3168e-05],
         [8.2669e-05, 5.7174e-05, 3.6212e-05,  ..., 2.2250e-05, 3.4547e-05,
          4.0065e-05],
         ...,
         [7.7786e-05, 5.1327e-05, 5.1337e-05,  ..., 2.4219e-05, 3.5704e-05,
          2.8052e-05],
         [5.6067e-05, 3.0498e-05, 3.0617e-05,  ..., 2.9279e-05, 2.0529e-05,
          6.7594e-05],
         [5.4029e-05, 6.1294e-05, 4.0449e-05,  ..., 1.9086e-05, 2.3124e-05,
          3.6273e-05]], grad_fn=<SoftmaxBackward0>))

## 执行训练

In [7]:
#执行训练
def train():
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(5):
        for i, data in enumerate(loader):
            loss, out = model(**data)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if i % 200 == 0:
                out = out.argmax(dim=1)
                acc = (out == data['label']).sum().item() / len(data['label'])
                print(epoch, i, len(loader), loss.item(), acc)


train()

0 0 1160 9.958338737487793 0.0
0 200 1160 9.95831298828125 0.0
0 400 1160 9.873355865478516 0.25
0 600 1160 9.574552536010742 0.5
0 800 1160 9.816194534301758 0.25
0 1000 1160 9.471392631530762 0.5
1 0 1160 9.954161643981934 0.0
1 200 1160 9.823026657104492 0.125
1 400 1160 9.803387641906738 0.25
1 600 1160 9.49328327178955 0.625
1 800 1160 9.527310371398926 0.5
1 1000 1160 9.695521354675293 0.25
2 0 1160 9.363061904907227 0.625
2 200 1160 9.483329772949219 0.625
2 400 1160 9.556771278381348 0.375
2 600 1160 9.676772117614746 0.25
2 800 1160 9.473091125488281 0.5
2 1000 1160 9.459622383117676 0.5
3 0 1160 9.854050636291504 0.125
3 200 1160 9.470598220825195 0.5
3 400 1160 9.407782554626465 0.75
3 600 1160 9.48289966583252 0.625
3 800 1160 9.481666564941406 0.5
3 1000 1160 9.61184310913086 0.375
4 0 1160 9.560114860534668 0.5
4 200 1160 9.721332550048828 0.25
4 400 1160 9.378260612487793 0.625
4 600 1160 9.243274688720703 0.75
4 800 1160 9.71529769897461 0.25
4 1000 1160 9.7167959213256

## 执行测试

In [19]:
#执行测试
def test():
    loader_test = torch.utils.data.DataLoader(dataset['test'],
                                              batch_size=8,
                                              shuffle=True,
                                              drop_last=True)

    correct = 0
    total = 0
    for i, data in enumerate(loader_test):
        with torch.no_grad():
            _, out = model(**data)

        out = out.argmax(dim=1)
        correct += (out == data['label']).sum().item()
        total += len(data['label'])

        print(i, len(loader_test), correct / total)

        if i == 5:
            break

    return correct / total


test()

0 144 0.5
1 144 0.4375
2 144 0.375
3 144 0.40625
4 144 0.4
5 144 0.4166666666666667


0.4166666666666667